# Armoria API

In [3]:
# auto-load when code changes outside
%load_ext autoreload
%autoreload 2
%load_ext pyinstrument

import sys
sys.path.insert(0,'..')


import os
from src.armoria_api import ArmoriaAPIPayload, ArmoriaAPIWrapper
from src.caption import Caption


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The pyinstrument extension is already loaded. To reload it, use:
  %reload_ext pyinstrument


# Examples of using the ArmoriaAPIWrapper 

In [4]:
coa_lion = {"t1":"or",
       "shield":"heater",
       "charges":[{"charge":"lionRampant","t":"gules","p":"e","size":1.5}],
       "ordinaries":[{"ordinary":"bordure","t":"azure"}],
      }

api = ArmoriaAPIWrapper(size = 500,format="png", coa=coa_lion)
api.show_image()

coa_cross = {"t1":"argent",
       "shield":"heater",
       "charges":[{"charge":"crossMoline",
                   "t":"azure",
                   "p":"e",
                   "size":1.5}]
      }

api = ArmoriaAPIWrapper(size = 500,format="png", coa=coa_cross)
api.show_image()

display-im6.q16: unable to open X server `' @ error/display.c/DisplayImageCommand/432.
display-im6.q16: unable to open X server `' @ error/display.c/DisplayImageCommand/432.


In [68]:
payload = {'t1': 'azure', 'shield': 'heater', 
             'charges': [{'charge': 'lionRampant', 't': 'argent', 'p': 'k', 'size': '0.7'},
                        {'charge': 'eagle', 't': 'or', 'p': 'n', 'size': '0.7'}],
             'ordinaries': []}
api = ArmoriaAPIWrapper(size = 500,format="png", coa=payload)
api.show_image()


In [4]:
payload = {'t1': 'or', 'shield': 'heater', 
             'charges': [{'charge': 'lionRampant', 't': 'or', 'p': 'abc', 'size': '0.3'},
                         {'charge': 'eagle', 't': 'azure', 'p': 'dfgz', 'size': '0.3'},
                         {'charge': 'crossHummetty', 't': 'argent', 'p': 'i', 'size': '0.3'}],
             'ordinaries': [{"ordinary":"bordure", "t":"azure"}]}

api = ArmoriaAPIWrapper(size = 500,format="png", coa=payload)
api.show_image()


In [35]:
# create a file to store all generated images files names and labels
import time

def ensure_dir(file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

def creat_caption_file(filename):
    f = open( filename,"w+")
    f.write('image,caption')
    f.write('\n')
    f.close()

def write_image_label_to_file(filename, line):
    with open(filename, 'a') as f:
        f.write(line)
        f.write('\n')
    f.close()

def generate_dataset(folder_name, caption_file, permutations, start_index=0):
    counter = start_index+1
    
    try:
        for i in range(start_index, len(permutations)):

            label = permutations[i]

            text_label = ' '.join(label).strip()

            struc_label = Caption(text_label,support_plural=True).get_structured()

            payload = ArmoriaAPIPayload(struc_label).get_armoria_payload()

            api = ArmoriaAPIWrapper(size = 500,format="png", coa=payload)

            sample_name = 'image_' + str(counter)

            image_full_path = folder_name + '/images/' + sample_name + '.png'

            ensure_dir(image_full_path)

            api.save_image(image_full_path)

            print('Image "{}" for label "{}" has been generated succfully' .format(image_full_path, text_label) )

            counter = i

            ## store image labels in on txt file (needed for data loader) 
            write_image_label_to_file(caption_file, sample_name + '.png,' + text_label)
            time.sleep(5)
    
    except Exception as e: 
        # keep retrying 
        print(e)
        time.sleep(60)
        generate_dataset(folder_name, caption_file, permutations, start_index=counter)


# Generate Permutations

In [6]:
import itertools
import src.alphabet as alphabet
from src.armoria_api import COLORS_MAP, SINGLE_LION_MODIFIERS_MAP, PLURAL_LION_MODIFIERS_MAP, CROSS_MODIFIERS_MAP, \
SINGLE_EAGLE_MODIFIERS_MAP, PLURAL_EAGLE_MODIFIERS_MAP, \
POSITIONS, SIZES, NUMBERS, NUMBERS_MULTI, SINGLE_POSITION

BORDER_MOD = ['& border', '']

## Single object

In [7]:

# lion, modifiers and colors
permutations1 = list(dict.fromkeys(itertools.product(list(COLORS_MAP.keys()), list(COLORS_MAP.keys()), list( SINGLE_LION_MODIFIERS_MAP.keys()),BORDER_MOD)))

# cross, modifiers and colors
permutations2 = list(dict.fromkeys(itertools.product(list(COLORS_MAP.keys()), list(COLORS_MAP.keys()), list( CROSS_MODIFIERS_MAP.keys()),BORDER_MOD)))

# eagle, modifiers and colors
permutations3 = list(dict.fromkeys(itertools.product(list(COLORS_MAP.keys()), list(COLORS_MAP.keys()), list( SINGLE_EAGLE_MODIFIERS_MAP.keys()),BORDER_MOD)))

possible_single_permutations = permutations1 + permutations2 + permutations3

print('Total number of permutations:', len(possible_single_permutations))

Total number of permutations: 720


## Plural Object with Number

In [8]:
# lion, modifiers and colors
permutations1 = list(dict.fromkeys(itertools.product(list(COLORS_MAP.keys()), list(COLORS_MAP.keys()), list(NUMBERS), list( PLURAL_LION_MODIFIERS_MAP.keys()),BORDER_MOD)))

# eagle, modifiers and colors
permutations2 = list(dict.fromkeys(itertools.product(list(COLORS_MAP.keys()), list(COLORS_MAP.keys()), list(NUMBERS), list( PLURAL_EAGLE_MODIFIERS_MAP.keys()),BORDER_MOD)))

possible_pl_permutations = permutations1 + permutations2

print('Total number of plural permutations:', len(possible_pl_permutations))


Total number of plural permutations: 4320


## Multi Objects - single

In [9]:

# lion & eagle
permutations1 = list(dict.fromkeys(itertools.product(list(COLORS_MAP.keys()),list(COLORS_MAP.keys()), list(COLORS_MAP.keys()), list( SINGLE_LION_MODIFIERS_MAP.keys()), list( SINGLE_EAGLE_MODIFIERS_MAP.keys()),BORDER_MOD)))

# lion & cross
permutations2 = list(dict.fromkeys(itertools.product(list(COLORS_MAP.keys()),list(COLORS_MAP.keys()), list(COLORS_MAP.keys()), list( SINGLE_LION_MODIFIERS_MAP.keys()), list( CROSS_MODIFIERS_MAP.keys()),BORDER_MOD)))

# eagle & cross
permutations3 = list(dict.fromkeys(itertools.product(list(COLORS_MAP.keys()),list(COLORS_MAP.keys()), list(COLORS_MAP.keys()), list( SINGLE_EAGLE_MODIFIERS_MAP.keys()), list( CROSS_MODIFIERS_MAP.keys()),BORDER_MOD)))

# lion & cross & eagle
permutations4 = list(dict.fromkeys(itertools.product(list(COLORS_MAP.keys()),list(COLORS_MAP.keys()),list(COLORS_MAP.keys()), list(COLORS_MAP.keys()), list( SINGLE_LION_MODIFIERS_MAP.keys()), list( SINGLE_EAGLE_MODIFIERS_MAP.keys()), list( CROSS_MODIFIERS_MAP.keys()),BORDER_MOD)))

possible_multi_single_permutations = permutations1 + permutations2 + permutations3 + permutations4

print('Total number of permutations:', len(possible_multi_single_permutations))


Total number of permutations: 91152


## Multi Objects - plural

In [10]:

# lion & eagle
permutations1 = list(dict.fromkeys(itertools.product(list(COLORS_MAP.keys()),list(COLORS_MAP.keys()), list(COLORS_MAP.keys()), list(NUMBERS_MULTI),list( PLURAL_LION_MODIFIERS_MAP.keys()), list(NUMBERS_MULTI), list( PLURAL_EAGLE_MODIFIERS_MAP.keys()),BORDER_MOD)))

# lion & cross
permutations2 = list(dict.fromkeys(itertools.product(list(COLORS_MAP.keys()),list(COLORS_MAP.keys()), list(COLORS_MAP.keys()), list(NUMBERS_MULTI),list( PLURAL_LION_MODIFIERS_MAP.keys()), list( CROSS_MODIFIERS_MAP.keys()),BORDER_MOD)))

# eagle & cross
permutations3 = list(dict.fromkeys(itertools.product(list(COLORS_MAP.keys()),list(COLORS_MAP.keys()), list(COLORS_MAP.keys()), list(NUMBERS_MULTI),list( PLURAL_EAGLE_MODIFIERS_MAP.keys()), list( CROSS_MODIFIERS_MAP.keys()),BORDER_MOD)))

# lion & cross & eagle
permutations4 = list(dict.fromkeys(itertools.product(list(COLORS_MAP.keys()),list(COLORS_MAP.keys()),list(COLORS_MAP.keys()), list(COLORS_MAP.keys()), list(NUMBERS_MULTI), list( PLURAL_LION_MODIFIERS_MAP.keys()), list(NUMBERS_MULTI),list( PLURAL_EAGLE_MODIFIERS_MAP.keys()), list( CROSS_MODIFIERS_MAP.keys()),BORDER_MOD)))

possible_multi_plural_permutations = permutations1 + permutations2 + permutations3 + permutations4

print('Total number of plural permutations:', len(possible_multi_plural_permutations))


Total number of plural permutations: 1081728


## Total number of permutations before adding border (588960)

## Total number of permutations with/out borders (1177920)

In [11]:
total_possible_permutations = possible_single_permutations + possible_pl_permutations + \
                            possible_multi_single_permutations  + possible_multi_plural_permutations
len(total_possible_permutations)

1177920

In [12]:
total_possible_permutations[0]

('A', 'A', 'lion', '& border')

## Total number of single permutations +  position and scale

In [33]:
FOLDER_NAME = '/home/space/datasets/COA/generated-data-api-large'
caption_file = FOLDER_NAME + '/' + 'captions.txt'
# FOLDER_NAME = '../generated'
creat_caption_file(caption_file)

In [ ]:
start_index = 0
generate_dataset(FOLDER_NAME,caption_file, total_possible_permutations, start_index)

Image "/home/space/datasets/COA/generated-data-api-large/images/image_1.png" for label "A A lion & border" has been generated succfully
Image "/home/space/datasets/COA/generated-data-api-large/images/image_0.png" for label "A A lion" has been generated succfully
Image "/home/space/datasets/COA/generated-data-api-large/images/image_1.png" for label "A A lion rampant & border" has been generated succfully
Image "/home/space/datasets/COA/generated-data-api-large/images/image_2.png" for label "A A lion rampant" has been generated succfully
Image "/home/space/datasets/COA/generated-data-api-large/images/image_3.png" for label "A A lion passt & border" has been generated succfully
Image "/home/space/datasets/COA/generated-data-api-large/images/image_4.png" for label "A A lion passt" has been generated succfully
Image "/home/space/datasets/COA/generated-data-api-large/images/image_5.png" for label "A A lion passt guard & border" has been generated succfully
Image "/home/space/datasets/COA/gen

Image "/home/space/datasets/COA/generated-data-api-large/images/image_59.png" for label "B A lion & border" has been generated succfully
Image "/home/space/datasets/COA/generated-data-api-large/images/image_60.png" for label "B A lion" has been generated succfully
Image "/home/space/datasets/COA/generated-data-api-large/images/image_61.png" for label "B A lion rampant & border" has been generated succfully
Image "/home/space/datasets/COA/generated-data-api-large/images/image_62.png" for label "B A lion rampant" has been generated succfully
Image "/home/space/datasets/COA/generated-data-api-large/images/image_63.png" for label "B A lion passt & border" has been generated succfully
Image "/home/space/datasets/COA/generated-data-api-large/images/image_64.png" for label "B A lion passt" has been generated succfully
Image "/home/space/datasets/COA/generated-data-api-large/images/image_65.png" for label "B A lion passt guard & border" has been generated succfully
Image "/home/space/datasets/

Image "/home/space/datasets/COA/generated-data-api-large/images/image_118.png" for label "B V lion head" has been generated succfully
Image "/home/space/datasets/COA/generated-data-api-large/images/image_119.png" for label "O A lion & border" has been generated succfully
Image "/home/space/datasets/COA/generated-data-api-large/images/image_120.png" for label "O A lion" has been generated succfully
Image "/home/space/datasets/COA/generated-data-api-large/images/image_121.png" for label "O A lion rampant & border" has been generated succfully
Image "/home/space/datasets/COA/generated-data-api-large/images/image_122.png" for label "O A lion rampant" has been generated succfully
Image "/home/space/datasets/COA/generated-data-api-large/images/image_123.png" for label "O A lion passt & border" has been generated succfully
Image "/home/space/datasets/COA/generated-data-api-large/images/image_124.png" for label "O A lion passt" has been generated succfully
Image "/home/space/datasets/COA/gener

In [ ]:
#Image "/home/space/datasets/COA/generated-data-api-large/images/image_507.png" for label "G A cross moline & border" has been generated succfully
# ProtocolError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


### Generate the last missing photo!

In [16]:
all_possible_s_permutations[10691]

(('A', 'G', 'cross patonce'), 'e', '0.7')

# Generate smaller dataset for local testing


In [51]:
counter = 10691
label = all_possible_permutations[10691]
label_ls = [*label[0], label[1], label[2]]
print(label_ls)
folder_name = '/home/space/datasets/COA/generated-data-api'
payload = ArmoriaAPIPayload(label_ls).get_armoria_payload()
print(payload)
api = ArmoriaAPIWrapper(size = 500,format="png", coa=payload)

sample_name = 'image_' + str(counter+1)

image_full_path = folder_name + '/images/' + sample_name + '.png'
text_label = ' '.join(label[0])

api.save_image(image_full_path)

print('Image "{}" for label "{}" has been generated succfully' .format(image_full_path, text_label) )


NameError: name 'all_possible_permutations' is not defined

In [8]:
all_possible_permutations = []

# lion, modifiers and colors
permutations1 = list(dict.fromkeys(itertools.product(['A'],list(COLORS_MAP.keys()), list( LION_MODIFIERS_MAP.keys()))))

# cross, modifiers and colors
permutations2 = list(dict.fromkeys(itertools.product(['A'], list(COLORS_MAP.keys()), list( CROSS_MODIFIERS_MAP.keys()))))

# eagle, modifiers and colors
permutations3 = list(dict.fromkeys(itertools.product(['A'], list(COLORS_MAP.keys()), list( EAGLE_MODIFIERS_MAP.keys()))))

all_possible_permutations = permutations1 + permutations2 + permutations3

print('Total number of permutations:', len(all_possible_permutations))
all_possible_permutations

Total number of permutations: 60


[('A', 'A', 'lion'),
 ('A', 'A', 'lion rampant'),
 ('A', 'A', 'lion passt'),
 ('A', 'A', 'lion passt guard'),
 ('A', 'A', "lion's head"),
 ('A', 'B', 'lion'),
 ('A', 'B', 'lion rampant'),
 ('A', 'B', 'lion passt'),
 ('A', 'B', 'lion passt guard'),
 ('A', 'B', "lion's head"),
 ('A', 'O', 'lion'),
 ('A', 'O', 'lion rampant'),
 ('A', 'O', 'lion passt'),
 ('A', 'O', 'lion passt guard'),
 ('A', 'O', "lion's head"),
 ('A', 'S', 'lion'),
 ('A', 'S', 'lion rampant'),
 ('A', 'S', 'lion passt'),
 ('A', 'S', 'lion passt guard'),
 ('A', 'S', "lion's head"),
 ('A', 'G', 'lion'),
 ('A', 'G', 'lion rampant'),
 ('A', 'G', 'lion passt'),
 ('A', 'G', 'lion passt guard'),
 ('A', 'G', "lion's head"),
 ('A', 'V', 'lion'),
 ('A', 'V', 'lion rampant'),
 ('A', 'V', 'lion passt'),
 ('A', 'V', 'lion passt guard'),
 ('A', 'V', "lion's head"),
 ('A', 'A', 'cross'),
 ('A', 'A', 'cross moline'),
 ('A', 'A', 'cross patonce'),
 ('A', 'B', 'cross'),
 ('A', 'B', 'cross moline'),
 ('A', 'B', 'cross patonce'),
 ('A', 'O'

In [18]:
FOLDER_NAME = 'generated-data-api-small'
generate_dataset(FOLDER_NAME, all_possible_permutations)

Image "generated-data-api-small/images/image_1.png" for label "A A lion" has been generated succfully
Image "generated-data-api-small/images/image_2.png" for label "A A lion rampant" has been generated succfully
Image "generated-data-api-small/images/image_3.png" for label "A A lion passt" has been generated succfully
Image "generated-data-api-small/images/image_4.png" for label "A A lion passt guard" has been generated succfully
Image "generated-data-api-small/images/image_5.png" for label "A A lion's head" has been generated succfully
Image "generated-data-api-small/images/image_6.png" for label "A B lion" has been generated succfully
Image "generated-data-api-small/images/image_7.png" for label "A B lion rampant" has been generated succfully
Image "generated-data-api-small/images/image_8.png" for label "A B lion passt" has been generated succfully
Image "generated-data-api-small/images/image_9.png" for label "A B lion passt guard" has been generated succfully
Image "generated-data-a